In [ ]:
import codecs
import tensorflow as tf
import os

from __future__ import print_function

import sys
import pandas as pd
import numpy as np
import re
import nltk


print(tf.__version__)

2.4.1


In [ ]:
INPUT_LENGTH = 10
OUTPUT_LENGTH = 10

In [ ]:
lines = open('/content/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('/content/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [ ]:
# создадим словарь, в котором соответсвующей строчке будет соответсвовать ее ID

id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [ ]:
# создадим список диалогов

convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [ ]:
# отсортируем предложения на вопрос/ответ

questions = []
answers = []
for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
print(len(questions))
print(len(answers))

221616
221616


In [ ]:
# создадим функцию для очистки текста

def clean_text(text):

    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|]", "", text)
    text = " ".join(text.split())

    return text

In [ ]:
# очистим текст

clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [ ]:
# найдем длины последовательностей

lengths = []

for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))

lengths = pd.DataFrame(lengths, columns=['counts'])

print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))

16.0
19.0
24.0
32.0


In [ ]:
# удалим вопросы и ответы длина которых больше 2 и меньше 16

min_line_length = 2
max_line_length = 16

short_questions_temp = []
short_answers_temp = []

for i, question in enumerate(clean_questions):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
        
short_questions = []
short_answers = []

for i, answer in enumerate(short_answers_temp):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
        
print(len(short_questions))
print(len(short_answers))

118928
118928


In [ ]:
nltk.download('punkt')

num_samples = 20000  # количество семплов для тренировки
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]

# получим токены для вопросов/ответов
short_questions_tok = [nltk.word_tokenize(sent) for sent in short_questions]
short_answers_tok = [nltk.word_tokenize(sent) for sent in short_answers]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# сделаем разбивку данных на тренировочные и валидационные

data_size = len(short_questions_tok)

training_input  = short_questions_tok[:round(data_size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] 
training_output = short_answers_tok[:round(data_size*(80/100))]

validation_input = short_questions_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] 
validation_output = short_answers_tok[round(data_size*(80/100)):]

print('training size', len(training_input))
print('validation size', len(validation_input))

training size 16000
validation size 4000


In [ ]:
# создадим словарь с частотой использования слов

vocab = {}
for question in short_questions_tok:
    for word in question:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers_tok:
    for word in answer:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1  

In [ ]:
# удалим редко используемые слова из словаря

threshold = 15
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [ ]:
print("Размер словаря:", len(vocab))
print("Количество используемых слов:", count)

Размер словаря: 12352
Количество используемых слов: 1310


In [ ]:
# создадим словарь, где каждому слову соответстует свой номер

WORD_CODE_START = 1
WORD_CODE_PADDING = 0


word_num  = 2
encoding = {}
decoding = {1: 'START'}
for word, count in vocab.items():
    if count >= threshold:
        encoding[word] = word_num 
        decoding[word_num ] = word
        word_num += 1

print("Количество используемых токенов:", word_num)

Количество используемых токенов: 1312


In [ ]:
# используем <UNK> токен для слов вне словаря

decoding[len(encoding)+2] = '<UNK>'
encoding['<UNK>'] = len(encoding)+2

dict_size = word_num+1
dict_size

1313

In [ ]:
# создадим функцию для приведения строк к единому вектору

def transform(encoding, data, vector_size=10):

    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                transformed_data[i][j] = encoding[data[i][j]]
            except:
                transformed_data[i][j] = encoding['<UNK>']
    return transformed_data

In [ ]:
# отформатируем тренировочные данные

encoded_training_input = transform(
    encoding, training_input, vector_size=INPUT_LENGTH)
encoded_training_output = transform(
    encoding, training_output, vector_size=OUTPUT_LENGTH)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)

encoded_training_input (16000, 10)
encoded_training_output (16000, 10)


In [ ]:
# отформатируем валидационные данные

encoded_validation_input = transform(
    encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(
    encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)

encoded_validation_input (4000, 10)
encoded_validation_output (4000, 10)


In [ ]:
# создадим модель encoder/decoder

INPUT_LENGTH = 10
OUTPUT_LENGTH = 10

encoder_input = tf.keras.layers.Input(shape=(INPUT_LENGTH,))
decoder_input = tf.keras.layers.Input(shape=(OUTPUT_LENGTH,))

encoder = tf.keras.layers.Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = tf.keras.layers.LSTM(512, unroll=True, return_sequences=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = tf.keras.layers.Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder = tf.keras.layers.LSTM(512, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

encoder KerasTensor(type_spec=TensorSpec(shape=(None, 10, 512), dtype=tf.float32, name=None), name='lstm/transpose_2:0', description="created by layer 'lstm'")
encoder_last KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")
decoder KerasTensor(type_spec=TensorSpec(shape=(None, 10, 512), dtype=tf.float32, name=None), name='lstm_1/transpose_2:0', description="created by layer 'lstm_1'")


In [ ]:
# реализуем механизм attention

from keras.layers import Activation, dot, concatenate, TimeDistributed, Dense

attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

output = TimeDistributed(Dense(512, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation="softmax"))(output)
print('output', output)

attention KerasTensor(type_spec=TensorSpec(shape=(None, 10, 10), dtype=tf.float32, name=None), name='attention/truediv:0', description="created by layer 'attention'")
context KerasTensor(type_spec=TensorSpec(shape=(None, 10, 512), dtype=tf.float32, name=None), name='dot_1/MatMul:0', description="created by layer 'dot_1'")
decoder_combined_context KerasTensor(type_spec=TensorSpec(shape=(None, 10, 1024), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")
output KerasTensor(type_spec=TensorSpec(shape=(None, 10, 1313), dtype=tf.float32, name=None), name='time_distributed_1/Reshape_1:0', description="created by layer 'time_distributed_1'")


In [ ]:
from keras.models import Model, load_model

model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 128)      168064      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 10, 512)      1312768     embedding[0][0]                  
______________________________________________________________________________________________

In [ ]:
# подготовим входы/выходы для encoder/decoder

training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = WORD_CODE_START
training_decoder_output = np.eye(dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = WORD_CODE_START
validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype('int')]

In [ ]:
model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          batch_size=64, epochs=50)

Epoch 1/50
250/250 [==============================] - 11s 21ms/step - loss: 0.0043 - val_loss: 0.0033
Epoch 2/50
250/250 [==============================] - 4s 18ms/step - loss: 0.0033 - val_loss: 0.0031
Epoch 3/50
250/250 [==============================] - 4s 18ms/step - loss: 0.0031 - val_loss: 0.0029
Epoch 4/50
250/250 [==============================] - 4s 18ms/step - loss: 0.0030 - val_loss: 0.0029
Epoch 5/50
250/250 [==============================] - 4s 18ms/step - loss: 0.0029 - val_loss: 0.0028
Epoch 6/50
250/250 [==============================] - 4s 18ms/step - loss: 0.0028 - val_loss: 0.0028
Epoch 7/50
250/250 [==============================] - 5s 19ms/step - loss: 0.0028 - val_loss: 0.0028
Epoch 8/50
250/250 [==============================] - 5s 18ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 9/50
250/250 [==============================] - 4s 18ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 10/50
250/250 [==============================] - 5s 18ms/step - loss: 0.0027 - val_l

In [ ]:
# создадим функции для предсказания и декодирования векторов в текст

def prediction(raw_input):
    clean_input = clean_text(raw_input)
    input_tok = [nltk.word_tokenize(clean_input)]
    input_tok = [input_tok[0][::-1]]  
    encoder_input = transform(encoding, input_tok, 10)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = WORD_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return output

def decode(decoding, vector):
    text = ''
    for i in vector:
        if i == 0:
            break
        text += ' '
        text += decoding[i]
    return text

In [ ]:
# проверим ответ модели на рандомном тексте

text_random = 'how are you?'

output2 = prediction(text_random)
decode(decoding, output2[0])

' been better . i read about that is been been'